In [1]:
import numpy as np
import itertools as it

import allel
import pandas as pd

from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

from datetime import datetime

import collections
def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

## The coalescent and sampling.


### Requirements

    - effective population size per generation;
    - number of generations;
    - scaled mutation rate.
    
   
### Index

#### I. Model assumptions.

#### II. Coalescent simulations.

#### III. Deployment across models. 

#### IV. Variance in count / frequency statistics.

#### V. Power analysis of shift ascertainment. 

#### VI. Window size

### I. Model assumptions.

**Demographics**

We will deal with terminal branches only. 

We're going to go with populations of species X, for which models have been proposed.
see model dict below. `pop` dictionaries contain model parameters per population specifc branches:

- `N`: Population effective size;
- `T`: Branch length


In [2]:
model_dict= {
        'schweinfurthii': {
            'N':10528, 
            'T':17925
        },
        'troglodytes': {
            'N': 72001, 
            'T': 17925
            },
        'ellioti': {
            'N': 6033,
            'T': 33402
        },
        'verus': {
            'N': 5710,
            'T': 33402
        }
}

**mutations** 

We re going to refer here to coalescent theory. Our objective will not concern statistics on observed data, but simulations. For this we will rely on the notebook [Simulations](https://nbviewer.jupyter.org/github/SantosJGND/Coalescent/blob/master/Simulations.ipynb), section **Algorithms for simulating sequence evolution**,  from this [repository](https://github.com/SantosJGND/Coalescent) focused on coalescent explorations in python.


To keep it simple we will consider in this notebook a single simulation. 

Given :
- time;
- effective population size;
- scaled mutation rate;

we draw the number of expected mutations emerging in our branch of interest until the present. 

## II. Simulations. Coalescent given number of samples.

We use a procedure of coalescent simulation given the number of samples. We extract per simulation the number of mutations simulated that are more recent than the MRCA of this branch. See the notebook above for a visual inspection of the output of the simulation procedure. We will use the function SimIII from that notebook, modified here for our purposes. 

We first chose a model from the dictionary above. 


In [3]:
pop_select= 'schweinfurthii'

We begin with a simple example: the number of private mutations expected expected in a sample of size `k` of haplotypes of size `seqL`, given an estimate of mutation rate per generation per base pair `muNuc`.

From this input we calculate the parameters `mu`, the expected mutation rate per generation and `theta`, the Watterson estimator of population genetic diversity. 

In [4]:
muNuc= 1.08e-8
seqL= 1e6

mu= muNuc * seqL
Ne= model_dict[pop_select]["N"]
k= 100 # Number of samples
Theta= 4 * Ne * mu


We now perform coalescent simulations. This returns a network containing coalescent and mutation events from present haplotypes to their MRCA (function `SimIII`). Each event has an associated time stamp. We use the function `get_PA` to extract the time stamps of mutation events more recent than the branch of interest. 

In [5]:
from tools.coal_sims import SimIII, get_PA

sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)

branch_len= model_dict[pop_select]['T']
MRCA= branch_len / Ne / 2

PA_obtain= get_PA(leaves,Ne,MRCA)
print('number of private alleles obtained: {}'.format(len(PA_obtain)))

number of private alleles obtained: 1781


#### get results across sample sizes

We now move to obtain results for a range of sample sizes. 

A number of repeats is performed per sample size, simulations are probablistic.

In [6]:
Nreps= 50
sample_range= [2,1000]
Nsteps= 20

range_use= np.linspace(*sample_range,Nsteps)
Nstudy_dict= {int(z):[] for z in range_use}

for si in range_use:
    k= int(si)
    for nr in range(Nreps):
        
        sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)
        
        PA_obtain= get_PA(leaves,Ne,MRCA)
        
        Nstudy_dict[k].append(len(PA_obtain))
        

The number of private alleles sampled is plotted against sample size. 

In [7]:
stats_dict= {
    z: {
        'mean': np.mean(g),
        'std': np.std(g)
    } for z,g in Nstudy_dict.items()
}

samp_order= sorted(Nstudy_dict.keys())

fig= [go.Scatter(
    x= samp_order,
    y= [stats_dict[x]['mean'] for x in samp_order],
    error_y= dict(
        array= [stats_dict[x]['std'] for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    )
)]

layout= go.Layout(
    title= pop_select,
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= 'private alleles'
    ),
    width= 950,
    height= 950
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)

### III. Deployment. Across populations:

We now scale this approach to cover all branches in `model_dict`. 

**common parameters**

In [8]:
### biological parameters
muNuc= 1.08e-8
seqL= 1e6
mu= muNuc * seqL

## sampling parameters
Nreps= 10
sample_range= [2,100]
Nsteps= 98
range_use= np.linspace(*sample_range,Nsteps,dtype= int)


**deployment**

In [9]:
pops_dict= {}
study_dict= {}

for pop_select in model_dict.keys():
    Ne= model_dict[pop_select]["N"]
    
    Theta= 4 * Ne * mu
    
    branch_len= model_dict[pop_select]['T']
    MRCA= branch_len / Ne / 2

    Nstudy_dict= {int(z):[] for z in range_use}

    for si in range_use:
        k= int(si)
        for nr in range(Nreps):

            sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)

            PA_obtain= get_PA(leaves,Ne,MRCA)

            Nstudy_dict[k].append(len(PA_obtain))
    
    stats_dict= {
        z: {
            'mean': np.mean(g),
            'std': np.std(g)
        } for z,g in Nstudy_dict.items()
    }
    samp_order= sorted(Nstudy_dict.keys())
    
    pops_dict[pop_select]= stats_dict
    study_dict[pop_select]= Nstudy_dict


**plot**

In [10]:
fig= [go.Scatter(
    x= samp_order,
    y= [pops_dict[pop][x]['mean'] for x in samp_order],
    error_y= dict(
        array= [pops_dict[pop][x]['std'] for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    name= pop
) for pop in model_dict.keys()]

layout= go.Layout(
    title= 'pop comparison',
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= '# of private alleles'
    ),
    height= 950,
    width= 950
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)

### IV. Variance in count / frequency statistics.

Some statistics of interest can depend on the number of mutations found in a sample. 

This is the case if mutations are characterised in any way, creating factors. If we are interested in ascertaining the true value of the size of each class in a population, than the variance in our observations are dependent on the number of samples. 

We investigate the impact of sampling haplotypes from model populations on estimator variance. 

We are going to assume that count numbers per sample size behave normally and extract posterior distributions from our simulated data this way. 


We begin by considering the variance around a single class. We set the expected proportion of that class in final counts to be `Pc`. We assum the proportion of cases of this class sampled will behave as a binomial variable of mean equals `Pc`, `n`  the sample size.


We extract two statistics:

- variance in proportion: `var_calc` and `var_samp`, obtained analytically and through sampling with `Nreps_II` replicates (bellow) under the assumptions stated above respectively.

- differences to expecation: `diffs`, the difference between expected and observed proportions when sampling.


#### i. Calc

Using data sampled above.

In [ ]:
from scipy.stats import norm
from scipy.stats import binom

Pc= 1 / 96
Nreps_II= 5

syndicate_dict= {
    z: {k: {} for k in g.keys()} for z,g in study_dict.items()
}

for pop_select in study_dict.keys():
    
    for si,sim_extract in study_dict[pop_select].items():
        size= int(si)
        
        pop_loc= pops_dict[pop_select][si]['mean']
        pop_scale= pops_dict[pop_select][si]['std']
        
        ## here the normal assumption is made
        sim_extract= norm.rvs(loc= pop_loc,scale= pop_scale,size= Nreps)
        sim_extract= np.array(sim_extract,dtype= int)
        sim_extract= sim_extract[sim_extract > 0]
        varA= []
        store_var= []
        store_diffs= []
        
        for nr in sim_extract:
            ## extract expected variance in proportion
            var_calc= nr * Pc * (1 - Pc)
            var_calc= np.sqrt(var_calc) / nr
            
            ## sample proportions assuming binomial behavior
            var_samp= binom.rvs(nr,Pc,size= Nreps_II)
            prop_vec= var_samp / nr
            
            ## proportion std and difference to expectation.
            prop_var= np.std(prop_vec)
            prop_diffs= Pc - prop_vec
            
            varA.append(var_calc)
            store_var.append(prop_var)
            store_diffs.extend(prop_diffs)
        
        syndicate_dict[pop_select][si]= {
            'var_calc': varA,
            'var_samp': store_var,
            'diffs': store_diffs
        }



#### ii. Plot. 

Interactive plots are not readable through NBviewer.

In [ ]:
def plot_dataSum(var_select, syndicate_dict, samp_order):

    fig= [go.Scatter(
        x= samp_order,
        y= [np.nanmean(syndicate_dict[pop][x][var_select]) for x in samp_order],
        error_y= dict(
            array= [np.nanstd(syndicate_dict[pop][x][var_select]) for x in samp_order],
            type= 'data',
            #symmetric= True,
            visible=True
        ),
        name= pop
    ) for pop in syndicate_dict.keys()]

    layout= go.Layout(
        title=  'stat = {}'.format(var_select),
        xaxis= dict(
            title= 'Nsamp'
        ),
        yaxis= dict(
            title= var_select
        )
    )

    Figure= go.Figure(data=fig, layout= layout)
    iplot(Figure)


In [13]:
## Var
var_select= 'var_calc'

plot_dataSum(var_select,syndicate_dict,samp_order)

In [14]:
## Var
var_select= 'var_samp'

plot_dataSum(var_select,syndicate_dict,samp_order)

In [15]:
## Var
var_select= 'diffs'

plot_dataSum(var_select,syndicate_dict,samp_order)

## V. Power analysis of shift ascertainment. 

From part one we find we describe how the number of private alleles depends on the number of samples available for a given length of region (1e6 in this example). 

In part III we describe how the variance in sample proportions evolves as a function of the number of samples, a function of the number of alleles sampled. 

In real world scenarios we might want to ascertain differences in proportion between populations. In this section the question becomes:

_How many individuals should we sample to have sufficient power to detect shifts in the proportion of classes of interest between populations?_

Assuming that the behaviour of the variable "# of alleles" is binomial, we estimate power as the proportion of counts from one population that fall within the distribution of another. 

One important thing to note is that power is not only impacted by the magnitude of the shift in class proportion, but also on the population in which it occurs.

For simplicity, we here combine results for sitations where the shifts happen in one or other population, by taking the minimum p-value of the two for every sample size combination.


In [11]:
pop_combs= it.combinations(pops_dict.keys(),2)
pop_combs= list(pop_combs)
pop_combs

[('schweinfurthii', 'troglodytes'),
 ('schweinfurthii', 'ellioti'),
 ('schweinfurthii', 'verus'),
 ('troglodytes', 'ellioti'),
 ('troglodytes', 'verus'),
 ('ellioti', 'verus')]

In [1]:
from scipy.stats import norm
from scipy.stats import binom

comb_select= ('schweinfurthii', 'ellioti')
pop_idx= {
    comb_select[idx]: idx for idx in range(len(comb_select))
}
alpha= 0.01

p= 1 / 192
Nreps= 100
Nsteps= 50
samp_order= np.linspace(*sample_range,Nsteps,dtype= int)
bi_select= it.product(samp_order,samp_order)
bi_select= list(bi_select)

rate_steps= 4
rate_max= 4
rate_range= np.linspace(2,rate_max,rate_steps,dtype= int)
rate_range= list(set(rate_range))
print('rates available:')
print(rate_range)

rate_stats= recursively_default_dict()

for bi in bi_select:
    for rate in rate_range:
        new_rate= p * rate
        rate_stats[bi][rate]= {
            x: [] for x in [*comb_select,'pval']
        }

        pops_probs= []
        for pop_skew in comb_select:

            pop_ref= [x for x in comb_select if x != pop_skew][0]

            Nskew= pops_dict[pop_skew][bi[pop_idx[pop_skew]]]['mean']
            Nskew_var= pops_dict[pop_skew][bi[pop_idx[pop_skew]]]['std']
            Nskew= norm.rvs(loc= Nskew,scale= Nskew_var, size= Nreps)
            #Nt= np.mean(Nt)
            Nref= pops_dict[pop_ref][bi[pop_idx[pop_ref]]]['mean']
            Nref_var= pops_dict[pop_ref][bi[pop_idx[pop_ref]]]['std']
            Nref= norm.rvs(loc= Nref,scale= Nref_var, size= Nreps)
            #Nref= np.mean(Nref)
            
            #Nskew= np.mean(Nskew)
            #print(Nt,Nref,Nskew)
            lbt= binom.ppf(alpha,Nskew,new_rate)

            probH2= binom.cdf(lbt,Nref, p)

            pops_probs.append(probH2)

        pops_probs= np.array(pops_probs)
        #print(pops_probs.shape)
        pops_probs= np.nanmean(pops_probs,axis= 1)

        if len(probH2):
            for idx in range(len(comb_select)):
                rate_stats[bi][rate][comb_select[idx]].append(bi[idx])

            rate_stats[bi][rate]['pval'].append(np.min(pops_probs))



NameError: name 'np' is not defined

In [129]:

def plot_power_3D(rate_stats,rate_range,rate= 2,func_select= np.nanmean,Nsteps= 4):
    bi_select= list(rate_stats.keys())
    
    fig= []
    d= 0
    
    colorscale= ['rgb(100,0,0)','rgb(100,0,100)','rgb(0,100,0)']
    colorscale= [[x / len(colorscale),colorscale[x]] for x in range(len(colorscale))]
    print(colorscale)
    for rate in rate_range:

        props= [(*bi,func_select(rate_stats[bi][rate]['pval'])) for bi in bi_select]
        props= np.array(props)
        #print(d)
        color_grad= np.zeros((Nsteps,Nsteps))
        color_grad+= d / len(rate_range)
        #color_grad= np.array(color_grad).reshape((Nsteps,Nsteps))
        #print(color_grad)
        fig.append(go.Surface(
            x= props[:,0].reshape((Nsteps,Nsteps)),
            y= props[:,1].reshape((Nsteps,Nsteps)),
            z= props[:,2].reshape((Nsteps,Nsteps)),
            cmin=0,
            cmax=1,
            cauto= False,
            showscale=False, opacity=1,
            surfacecolor= color_grad,
            colorscale= colorscale
        ))
        d += 1

    layout= go.Layout()

    Figure= go.Figure(data= fig)
    #Figure.show()
    iplot(Figure)



In [130]:
np.prod((Nsteps,Nsteps))

2500

In [131]:
import plotly.graph_objects as go

plot_power_3D(rate_stats,rate_range,Nsteps= Nsteps)

[[0.0, 'rgb(100,0,0)'], [0.3333333333333333, 'rgb(100,0,100)'], [0.6666666666666666, 'rgb(0,100,0)']]


In [19]:

def plot_power(rate_stats,rate= 2,func_select= np.nanmean):
    bi_select= list(rate_stats.keys())
    props= [(*bi,func_select(rate_stats[bi][rate]['pval'])) for bi in bi_select]
    props= np.array(props)


    fig= [go.Scatter(
        x= props[:,0],
        y= props[:,1],
        mode= 'markers',
        marker= {
            'color': props[:,2],
            "colorbar": dict(
                title="Colorbar"
            ),
            'colorscale': 'Viridis',
                'line': {'width': 0},
                'size':6,
                'symbol': 'circle',
            "opacity": 1
        }
    )]

    layout= go.Layout(
        title= 'rate: {}'.format(rate),
        xaxis= dict(
            title= comb_select[0]
        ),
        yaxis= dict(
            title= comb_select[1]
        ),
        height= 800,
        width= 800
    )

    Figure= go.Figure(data= fig,layout= layout)

    iplot(Figure)

rate= 4
plot_power(rate_stats,rate= rate)

**Fig. pval comparison** power to detect changes in class proportions between two populations for sequences of length `seqL` across sample sizes.

In [20]:
rate= 12
plot_power(rate_stats,rate= rate,func_select= np.nanmean)


In [30]:
#### ii. together

border_pval= .99 # lower power threshold

bi_select= list(rate_stats.keys())

props= []
func_select= np.nanmean

for bi in bi_select:
    rate_num= [(rate,func_select(rate_stats[bi][rate]['pval'])) for rate in rate_range]
    rate_sum= [x[0] for x in rate_num if x[1] > border_pval]
    
    if not rate_sum: 
        continue
    else:
        rate_sum= min(rate_sum)
        
    props.append((*bi,rate_sum))

props= np.array(props)

props_dict= {
    z: [x for x in range(props.shape[0]) if props[x,2] == z] for z in list(set(props[:,2]))
}
props_dict= {
    z: props[g,:] for z,g in props_dict.items()
}

fig= [go.Scatter3d(
    x= props_dict[z][:,0],
    y= props_dict[z][:,1],
    z= props_dict[z][:,2],
    mode= 'markers',
    name= 'min rate: ' + str(z),
    marker= dict(
        size= 8
    )
) for z in sorted(list(props_dict.keys()))]

fig_borders.extend(fig)

layout= go.Layout(
    title= 'rate: {}'.format(rate),
    xaxis= dict(
        title= comb_select[0]
    ),
    yaxis= dict(
        title= comb_select[1]
    ),
    height= 800,
    width= 800
)

Figure= go.Figure(data= fig_borders,layout= layout)
iplot(Figure)

**Figure Rate** Minimum shift in mutation probability identifiable by sample size combination for the two populations, for regions 1M in length. 

## VI. Window size

Given what we now know, we are ready to address a practical problem in real world analysis, by reversing the question: 

_What is the minimum sequence length for acceptable ascertaining power in comparing two populations when we are constrained in the number of samples?_

i.e. 

_Estimating optimal genomic window size._

In [31]:
pop_size= {
        'schweinfurthii': 6,
        'troglodytes': 4,
        'ellioti': 10,
        'verus': 5
}

In [34]:
### biological parameters
muNuc= 1.08e-8

## sampling parameters
Nreps= 10
Lseq_range= [1e3,1e8]
Nsteps= 100
range_use= np.linspace(*Lseq_range,Nsteps,dtype= int)


In [35]:

pops_dict= {}
study_dict= {}
for pop_select in model_dict.keys():
    print(pop_select)
    Ne= model_dict[pop_select]["N"]
    
    branch_len= model_dict[pop_select]['T']
    MRCA= branch_len / Ne / 2

    Nstudy_dict= {z:[] for z in range_use}
    
    for seqL in range_use:
        
        mu= muNuc * seqL
        Theta= 4 * Ne * mu
        
        k= int(pop_size[pop_select])
        for nr in range(Nreps):

            sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)

            PA_obtain= get_PA(leaves,Ne,MRCA)

            Nstudy_dict[seqL].append(len(PA_obtain))
    
    stats_dict= {
        z: {
            'mean': np.mean(g),
            'std': np.std(g)
        } for z,g in Nstudy_dict.items()
    }
    samp_order= sorted(Nstudy_dict.keys())
    
    pops_dict[pop_select]= stats_dict
    study_dict[pop_select]= Nstudy_dict


schweinfurthii
troglodytes
ellioti
verus


In [36]:
from scipy.stats import binom
comb_select= ('schweinfurthii', 'ellioti')
pop_idx= {
    comb_select[idx]: idx for idx in range(len(comb_select))
}
alpha= 0.01

p= 1 / 192
Nreps= 1000


rate_steps= 50
rate_max= 5
rate_range= np.linspace(1,rate_max,rate_steps)
rate_range= list(set(rate_range))
#print('rates available:')
#print(rate_range)

rate_stats= recursively_default_dict()

for bi in range_use:
    for rate in rate_range:
        new_rate= p * rate
        rate_stats[rate][bi]= {
            x: [] for x in [*comb_select,'pval']
        }

        pops_probs= []
        for pop_skew in comb_select:

            pop_ref= [x for x in comb_select if x != pop_skew][0]

            Nskew= pops_dict[pop_skew][bi]['mean']
            Nskew_var= pops_dict[pop_skew][bi]['std']
            Nskew= norm.rvs(loc= Nskew,scale= Nskew_var, size= Nreps)
            #Nt= np.mean(Nt)
            Nref= pops_dict[pop_ref][bi]['mean']
            Nref_var= pops_dict[pop_ref][bi]['std']
            Nref= norm.rvs(loc= Nref,scale= Nref_var, size= Nreps)
            #Nref= np.mean(Nref)
            
            #Nskew= np.mean(Nskew)

            #print(Nt,Nref,Nskew)
            lbt= binom.ppf(alpha,Nskew,new_rate)

            probH2= binom.cdf(lbt,Nref, p)

            pops_probs.append(probH2)

        pops_probs= np.array(pops_probs)
        #print(pops_probs.shape)
        pops_probs= np.nanmean(pops_probs,axis= 1)

        if len(probH2):
            for idx in range(len(comb_select)):
                rate_stats[rate][bi][comb_select[idx]].append(bi)

            rate_stats[rate][bi]['pval'].append(np.min(pops_probs))



In [37]:
nbins= 20

fig= [go.Scatter(
    x= range_use,
    y= [rate_stats[rate][x]['pval'][0] for x in range_use],
    mode= "markers",
    name= str(rate)
) for rate in sorted(rate_range[::2])]

layout= go.Layout(
    title=  'combo: {} - {}'.format(*comb_select),
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= 'p-value'
    )
)

Figure= go.Figure(data= fig, layout= layout)
iplot(Figure)

In [55]:
p_threshold= .99

rate_min= {
    rate: [x for x in rate_stats[rate].keys() if len(rate_stats[rate][x]) > 1] for rate in rate_range
}

rate_min= {
    rate: [x for x in g if np.nanmean(rate_stats[rate][x]['pval']) > p_threshold] for rate,g in rate_min.items()
}

rate_min= {z:g for z,g in rate_min.items() if len(g) > 1}


rate_min= {
    z:min([x for x in g if x != 1000]) for z,g in rate_min.items()
}

rate_mink= list(sorted(rate_min.keys()))

fig= [go.Scatter(
    x= rate_mink,
    y= [rate_min[x] for x in rate_mink]
)]

layout= go.Layout(
    title=  'combo: {} - {}'.format(*comb_select),
    xaxis= dict(
        title= 'mutation proportion fold change'
    ),
    yaxis= dict(
        title= 'min window size'
    )
)

Figure= go.Figure(data= fig, layout= layout)
iplot(Figure)

In [54]:
rate_stats[4.26530612244898][2021181]

{'schweinfurthii': [2021181],
 'ellioti': [2021181],
 'pval': [0.8640390585001969]}